In [1]:
### This will attempt to run randomise on LARGE data sets..
# coding: utf-8
import nipype.pipeline.engine as pe
import nipype.interfaces.utility as niu
import nipype.interfaces.fsl as fsl
import os
import nibabel as nib
import scipy
import numpy as np
import glob
from scipy.sparse import coo_matrix
from numpy import genfromtxt

In [2]:
## Coordinate encoded sparse matrix...
sampleDotFile = '/home/dagutman/data/WholeBrainMatrixConnectivity_downsample_4/Fox_C_fdt_matrix3_downsample_4.dot'

### don't forget the coords file!!
# -rw-r--r--  1 ehecht cci-users 109K Apr 13  2015 coords_for_fdt_matrix3
# -rw-r--r--  1 ehecht cci-users  41K Apr 13  2015 fdt_paths.nii.gz
# -rw-r--r--  1 ehecht cci-users  12M Apr 21  2015 Fox_C_fdt_matrix3_downsample_4.dot
# -rw-r--r--  1 ehecht cci-users  15M Apr 21  2015 Fox_D_fdt_matrix3_downsample_4.dot
# -rw-r--r--  1 ehecht cci-users  12M Apr 21  2015 Fox_E_fdt_matrix3_downsample_4.dot
# -rw-r--r--  1 ehecht cci-users 7.2M Apr 21  2015 Fox_F_fdt_matrix3_downsample_4.dot
# -rw-r--r--  1 ehecht cci-users  12M Apr 21  2015 Fox_G_fdt_matrix3_downsample_4.dot
# -rw-r--r--  1 ehecht cci-users  11M Apr 21  2015 Fox_H_fdt_matrix3_downsample_4.dot
#my_data[999]  ## numpy is y,x  or row by column

## CREATING A MASK
mask A, B, C, D E and F

most efficient way to get all NON ZERO's across all matrixes'

Synthetic data sets--  want to have subject in one group and put in the UPPER LEFT CORNER

BOTTOM RIGHT CORNER vs TOP LEFT CORNER

display arrays in python larger than 4 x 4

DO it very small

In [2]:
def convertDotFileToNifti( dotFileIn, NIIout, reShapeByPofTwo=False, debug=False):
    ### Eventually instead of outputting a linear gigantic X,Y matric I am going to reshape
    ### it and take advantage of the z Plane
    ## This is a sparsely encoded matrix of X by Y by Connectivity generated by probtrackX
    sparseCoordData = genfromtxt(sampleDotFile, delimiter='  ')
    
    ## Split out the X, Y and C values from the dot file
    x_array = my_data[:,0]
    y_array = my_data[:,1]
    c_array = my_data[:,2]   ## Connectivity strength matrix
    
    width=  int(x_array.max())  ## Must remember numpy is y,x not x,y
    height = int(y_array.max())

    ## Coordinates are 1 indexed from the original file, but the created matrix is
    ## 0 indexed, so need to adjust accordingly
    indexArray = y_array * (width+1) + x_array
    indexArray = indexArray.astype(int)

    if debug: print width,height,my_data.shape,a.shape
    
    # Create a placeholder array in the proper shape to hold the unsparsed data matrix
    a = np.zeros( (1,   (width+1) * (height+1) ))
    
    ### Copy the connectivity numbers in a single operation using a linearly indexed Array
    a[0,indexArray] = c_array
    
    ## Now reshape the array into height, width  since numpy is y,x ordered
    a = np.reshape(a, (height+1, width+1))

    ## probably a more efficient way to convert this from a 2D to a 3D array,
    
    if reShapeByPofTwo:
        niiMatrix = np.zeros((a.shape[0]/8,a.shape[1]/8,64))
        a = np.reshape(a,(5264/8,5264/8,64))
        niiMatrix = a ## Copy the data from A into the nifti matrix
        print "Reshaping Matrix"

    else:
        niiMatrix = np.zeros((a.shape[0],a.shape[1],1))  ## make this into a 3-D Matrix
        niiMatrix[:,:,0] = a ## Copy the data from A into the nifti matrix
    
    niiImg = nib.Nifti1Image(niiMatrix, np.eye(4))
    
    ### I need to reshape A as well
##    print np.reshape(testDiag, (4,4,4)).shape
    myNiiImg.to_filename(NIIout)
    print "File Saved!"
    return True

In [ ]:
dotFiles = glob.glob('/home/dagutman/data/WholeBrainMatrixConnectivity_downsample_4/*.dot')
for df in dotFiles:
    print df
    convertDotFileToNifti( df, df+'.nii.gz',reShapeByPofTwo=True)

In [ ]:
curMatrixShape = a.shape
print curMatrixShape
print niiMatrix.shape
## np.eye is the identity matrix for affine transformation
myNiiImg = n
# nibabel.save(niiMatrix,'testMyNII.nii.gz')

In [ ]:
myArray = my_data
xy_array = myArray[:, :2]
v_array = myArray[:, 2]
### need to compute maxX, maxY
# calculate the linear  index of this i*whatever + j
a = np.zeroes( ())

In [ ]:
import primefac
for x in primefac.primefac(5264):
    print x

## Split by the biggest factor of two that will factor this matrix?/
# 2
# 2
# 2
# 2
# 7
# 47

In [ ]:
### Input Data Sets consist of .dot files which we need to eventually convert into nifti files
# M is the "real" data with size 5263x5263

# Z=M*0; % creates 5263x5263 matrix of zeros

# N=cat(3,M,Z); % concatenate M and Z in the 3rd dimension


# > in matlab
# >
# > x=load('fdt_matrix3.dot');
# > M=full(spconvert(x));
# >
# > % reshape matrix into 3 dimensions - need to do math to find factors
# > B=reshape(M,64,529,17161);
# >
# > % need this toolbox -
# > https://www.mathworks.com/matlabcentral/fileexchange/8797-tools-for-nifti-and-analyze-image?focused=3806902&tab=function
# >
# > save('connectivity_matrix_nothr_reshape3d.mat','B','-v7.3');
# > nii=make_nii(B);
# > save_nii(nii,'connectivity_matrix_nothr_reshape3d.nii');
